# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

## Reading the data from csv file

In [69]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [70]:
# Loading data from csv file
import pandas as pd
data_path = '/gdrive/MyDrive/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [71]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [72]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [73]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [74]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

In [76]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tf_idf_vect = TfidfVectorizer(ngram_range=(2,3), min_df=10, max_features=4000)
X_train = tf_idf_vect.fit_transform(train_data['text'])
test_data = tf_idf_vect.transform(test_data['text'])

x = [[]] * 5
lst1 = []
tup = ()
lst2 = []
dict1 = {}
x_i=0
lst3 =[]
np.random.seed(0)
for i in range(5):
    x[i] = list((np.random.normal(0,1,4000)))

vec = list(X_train.toarray())

for i in range(len(vec)):
    tup = ()
    lst1 = []
    lst = []
    for j in range(len(x)):
        value=0
        for k in range(len(x[0])):
            value = value + vec[i][k] * x[j][k] 
        if value > 0:
            lst1.append(1)
        else:
            lst1.append(-1)
    tup = tuple(lst1)
    if tup in dict1.keys():
        lst=[]
        lst = dict1.get(tup)
        lst.append(i)
        dict1[tup] = lst
    else:
        lst3=[]
        lst3.append(i)
        dict1[tup] = lst3
    

In [83]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

import numpy as np

def predictLabels (test_data):

    vec_test = list(test_data.toarray())
    dict2 = {}
    for i in range(len(vec_test)):
        tup = ()
        lst1 = []
        lst = []
        for j in range(len(x)):
            value=0
            for k in range(len(x[0])):
                value = value + vec_test[i][k] * x[j][k] 
            if value > 0:
                lst1.append(1)
            else:
                lst1.append(-1)
        tup = tuple(lst1)
        if tup in dict1.keys():
            lst=[]
            lst = dict1.get(tup)
            dict2[i] = lst

    dict3={}
    dist=0
    for i in range(len(vec_test)):
        lst = []
        lst1=[]
        lst = dict2.get(i)
        for j in range(len(lst)):
            dist = 1 - (dot(vec_test[i],vec[lst[j]])/norm(vec_test[i])*norm(vec[lst[j]]))
            lst1.append(dist)
        dict3[i] = lst1    

    for i in range(len(vec_test)):
        lst = []
        dist_sorted = []
        lst = dict3.get(i)
        lst=np.array(lst)
        dist_sorted = list(np.argsort(lst))
        dist_sorted = dist_sorted[0:11]
        lst4=[]
        lst4 = dict2.get(i)
        lst5=[]
        for j in range(11):
            val = lst4[dist_sorted[j]]
            lst5.append(val)
        dict3[i] = lst5
    
    lst2=[]
    for i in range(10):
        lst=[]
        lst = dict3.get(i)
        lst1=[]
        for j in range(11):
            lst1.append(train_data.iloc[lst[j]]['category'])
        df = pd.DataFrame(lst1)
        val = df.value_counts().idxmax()
        lst2.append(val)

    return(lst2) 


## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [84]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


******** Success ******** Accuracy Achieved =  200 %
